# Open MP Study

need the execution of the file src/compilation.ipynb

## get the results


In [1]:
import os
import subprocess

import experimentsDef as expDef

import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [5]:
implementation = expDef.getImplementationDetails()

experimentsHandDetails = expDef.getExpDetails()

relevantCodeVersion = ["openMPMutationV1", "baseline", "openMPMutationV2"]

experimentsRelevantDetails = {
    "perDefaultCPU" : {
        "w" : 32,
        "h" : 32,
        "g" : 5000,
        "m" : 0.00001,
    },
    "chargeCPU" : {
        "w" : 128,
        "h" : 128,
        "g" : 10000,
        "m" : 0.00001,
    },
}

columnsCSV = ["Gen","Resource","Stamp","TimeStamp_Start","TimeStamp_End","Duration","Indiv_id", "exec", "w", "h", "g", "mutation_rate", "versionCode", "versionExp"]


In [6]:
aggregate = pd.DataFrame(data=None, index=None, columns=columnsCSV)
workDir = os.getcwd()

for version in relevantCodeVersion:
    for exp in experimentsRelevantDetails:
        path = os.path.join(workDir, "simulations", version, exp, "trace.csv")
        data = pd.read_csv(path, sep=',')
        data["exec"] = "nameCPU"
        data["w"] = experimentsRelevantDetails[exp]["w"]
        data["h"] = experimentsRelevantDetails[exp]["h"]
        data["g"] = experimentsRelevantDetails[exp]["g"]
        data["mutation_rate"] = experimentsRelevantDetails[exp]["m"]
        data["versionCode"] = version
        data["versionExp"] = exp
        aggregate = aggregate.append(data, ignore_index=True)


## compare the results

In [7]:
df = aggregate[aggregate['Stamp'] == "STEP"]
df["Duration"] = df["Duration"].astype(float)
dfExp = df.groupby(["versionCode", "versionExp"]).mean().reset_index()
print(dfExp)

        versionCode     versionExp    Duration  mutation_rate
0          baseline      chargeCPU  56228247.3        0.00001
1          baseline  perDefaultCPU   3819182.3        0.00001
2  openMPMutationV1      chargeCPU  17510034.8        0.00001
3  openMPMutationV1  perDefaultCPU   2802235.0        0.00001
4  openMPMutationV2      chargeCPU  15351972.5        0.00001
5  openMPMutationV2  perDefaultCPU   2101269.8        0.00001


In [8]:
# get the ratio with baseline for each experiment

def getRatio(df, versionExp):
    dfIn = df[(df["versionExp"] == versionExp) & (df["versionCode"] == "baseline")]
    return dfIn["Duration"].values[0]
    

dfExp["ratio"] = dfExp.apply(lambda row : getRatio(dfExp, row["versionExp"])/row["Duration"], axis=1)

print(dfExp)

        versionCode     versionExp    Duration  mutation_rate     ratio
0          baseline      chargeCPU  56228247.3        0.00001  1.000000
1          baseline  perDefaultCPU   3819182.3        0.00001  1.000000
2  openMPMutationV1      chargeCPU  17510034.8        0.00001  3.211201
3  openMPMutationV1  perDefaultCPU   2802235.0        0.00001  1.362906
4  openMPMutationV2      chargeCPU  15351972.5        0.00001  3.662607
5  openMPMutationV2  perDefaultCPU   2101269.8        0.00001  1.817559
